In [1]:
from sklearn.datasets import load_iris, fetch_20newsgroups, load_boston
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
li = load_iris()

print("获取特征值")
print(type(li.data))
print(li.data)
print(li.data.shape)
print("目标值")
print(li.target)
print('-' * 50)
print(li.DESCR)
print('-' * 50)
print(li.feature_names)  # 重点
print('-' * 50)
print(li.target_names)
print('-' * 50)


获取特征值
<class 'numpy.ndarray'>
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 

In [ ]:
# 注意返回值, 训练集 train  x_train, y_train ；测试集  test   x_test, y_test
# 默认是乱序的,random_state为了确保两次的随机策略一致，往往会带上
x_train, x_test, y_train, y_test = train_test_split(li.data, li.target, test_size=0.25, random_state=1)

print("训练集特征值和目标值：", x_train, y_train)
print("训练集特征值shape", x_train.shape)

print("测试集特征值和目标值：", x_test, y_test)
print("测试集特征值shape", x_test.shape)

# 分类估计器

In [3]:
# K近邻
"""
K-近邻预测用户签到位置
:return:None
"""
data = pd.read_csv("D:\wow\python study\wd_python\培训\笔记\ml\python_ml\data\FBlocation/train.csv")

print(data.head(10))
print(data.shape)
# 处理数据
# 1、缩小数据,查询数据,为了减少计算时间
data = data.query("x > 1.0 &  x < 1.25 & y > 2.5 & y < 2.75")

# 处理时间的数据
# print(data['time'])
time_value = pd.to_datetime(data['time'], unit='s')

print(time_value)  #最大时间是1月10号
#
# 把日期格式转换成 字典格式，把年，月，日，时，分，秒转换为字典格式，
time_value = pd.DatetimeIndex(time_value)
#
print('-' * 50)
print(time_value)
print('-' * 50)
# 构造一些特征，执行的警告是因为我们的操作是复制，loc是直接放入
print(type(data))
# data['day'] = time_value.day
# data['hour'] = time_value.hour
# data['weekday'] = time_value.weekday
#日期，是否是周末，小时对于个人行为的影响是较大的
data.insert(data.shape[1], 'day', time_value.day)
data.insert(data.shape[1], 'hour', time_value.hour)
data.insert(data.shape[1], 'weekday', time_value.weekday)

#
# 把时间戳特征删除
data = data.drop(['time'], axis=1)
print('-' * 50)
print(data)
#
# # 把签到数量少于n个目标位置删除
place_count = data.groupby('place_id').count()
# # 把index变为0,1,2，3,4,5,6这种效果，从零开始拍，原来的index是地点
#只选择去的人大于3的数据，认为1,2,3的是噪音
tf = place_count[place_count.row_id > 3].reset_index()
# 根据设定的地点目标值，对原本的样本进行过滤
data = data[data['place_id'].isin(tf.place_id)]
print(data)
# # 取出数据当中的特征值和目标值
y = data['place_id']
# 删除目标值，保留特征值，
x = data.drop(['place_id'], axis=1)
# 删除无用的特征值
x = x.drop(['row_id'], axis=1)
print(x.shape)
print(x.columns)
# 进行数据的分割训练集合测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)

# 特征工程（标准化）,下面3行注释，一开始我们不进行标准化，看下效果，目标值要不要标准化？
std = StandardScaler()
# #
# # # 对测试集和训练集的特征值进行标准化,服务于knn fit
x_train = std.fit_transform(x_train)
# # transform返回的是copy，不在原有的输入对象中去修改
# print(id(x_test))
print(std.mean_)
print(std.var_)
x_test = std.transform(x_test)  #transfrom不再进行均值和方差的计算，是在原有的基础上去标准化
print('-' * 50)
# print(id(x_test))
print(std.mean_)
print(std.var_)
# # 进行算法流程 # 超参数，可以通过设置n_neighbors=5，来调整结果好坏
knn = KNeighborsClassifier(n_neighbors=5)

# # fit， predict,score，训练
knn.fit(x_train, y_train)
# # #
# # # 得出预测结果
y_predict = knn.predict(x_test)
# #
print("预测的目标签到位置为：", y_predict)
# # #
# # # # 得出准确率
print("预测的准确率:", knn.score(x_test, y_test))

   row_id       x       y  accuracy    time    place_id
0       0  0.7941  9.0809        54  470702  8523065625
1       1  5.9567  4.7968        13  186555  1757726713
2       2  8.3078  7.0407        74  322648  1137537235
3       3  7.3665  2.5165        65  704587  6567393236
4       4  4.0961  1.1307        31  472130  7440663949
5       5  3.8099  1.9586        75  178065  6289802927
6       6  6.3336  4.3720        13  666829  9931249544
7       7  5.7409  6.7697        85  369002  5662813655
8       8  4.3114  6.9410         3  166384  8471780938
9       9  6.3414  0.0758        65  400060  1253803156
(29118021, 6)
600          65380
957         785470
4345        400082
4735        514983
5580        732410
             ...  
29100203     38036
29108443    602524
29109993    658994
29111539    262421
29112154    687667
Name: time, Length: 17710, dtype: int64
600        1970-01-01 18:09:40
957        1970-01-10 02:11:10
4345       1970-01-05 15:08:02
4735       1970-01-06 23:03:

In [4]:
# # 构造一些参数的值进行搜索
param = {"n_neighbors": [3, 5, 10, 12, 15]}
#
# 进行网格搜索，cv=3是3折交叉验证，用其中2折训练，1折验证
gc = GridSearchCV(knn, param_grid=param, cv=3)

gc.fit(x_train, y_train)

print("在测试集上准确率：", gc.score(x_test, y_test))

print("在交叉验证当中最好的结果：", gc.best_score_)

print("选择最好的模型是：", gc.best_estimator_)

print("每个超参数每次交叉验证的结果：", gc.cv_results_)

C:\Program Files\Python39\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


在测试集上准确率： 0.47966903073286055
在交叉验证当中最好的结果： 0.4596467484726366
选择最好的模型是： KNeighborsClassifier(n_neighbors=10)
每个超参数每次交叉验证的结果： {'mean_fit_time': array([0.0163033 , 0.01794362, 0.01438554, 0.01496498, 0.01429184]), 'std_fit_time': array([0.00047315, 0.0024538 , 0.0011315 , 0.0014153 , 0.00047183]), 'mean_score_time': array([0.17185942, 0.18119041, 0.19017363, 0.2217358 , 0.22039278]), 'std_score_time': array([0.00775007, 0.01447412, 0.00543769, 0.01513676, 0.0042397 ]), 'param_n_neighbors': masked_array(data=[3, 5, 10, 12, 15],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 3}, {'n_neighbors': 5}, {'n_neighbors': 10}, {'n_neighbors': 12}, {'n_neighbors': 15}], 'split0_test_score': array([0.44468085, 0.4607565 , 0.46170213, 0.45650118, 0.45508274]), 'split1_test_score': array([0.43390873, 0.45660913, 0.45542681, 0.45329865, 0.44809648]), 'split2_test_score': array([0.43982029, 0.45684559, 0.4618113 , 0.45